# Crash Course in Supercomputing: Compute Pi in a Serial Fashion


Adapted from: https://github.com/NERSC/crash-course-supercomputing/tree/main/darts-suite/fortran

## This program demonstrates computing $\pi$ in Fortran using the Monte Carlo method in a Serial Fashion:

### In file computing_pi_serial.f90

```{literalinclude} code/app/compute_pi_serial.f90
---
language: fortran
---
```

### In the file lcgenerator_mod.f90

```{literalinclude} code/src/lcgenerator_mod.f90
---
language: fortran
---
```

### In the file get_nr_iters_mod.f90

```{literalinclude} code/src/get_nr_iters_mod.f90
---
language: fortran
---
```

## Code Analysis by Grok 4

### Overview of the Code

This Fortran code is a complete, self-contained program designed to approximate the value of π (pi) using the Monte Carlo simulation method. The approach is serial (non-parallel), meaning it runs sequentially on a single processor without any concurrency. The program generates pseudo-random points within a unit square and counts how many fall inside a quarter-circle inscribed in that square. The ratio of points inside the circle to the total points, multiplied by 4, provides an estimate of π.

The code is structured as a main program (`compute_pi_serial`) and two supporting modules:
- `get_nr_iters_mod`: Handles reading the number of iterations (trials) from command-line arguments or uses a default value.
- `lcgenerator_mod`: Provides a simple linear congruential generator (LCG) for pseudo-random number generation.

The code uses modern Fortran features (e.g., from Fortran 2008/2018 standards), including intrinsic modules like `iso_fortran_env` for portable data types (e.g., 64-bit reals and integers). It employs double-precision arithmetic for accuracy and includes error handling for input.

I'll explain the code in detail, breaking it down by section: first the main program, then each module. I'll cover the purpose, syntax, variables, logic, and any key Fortran concepts.

### Main Program: `compute_pi_serial`

This is the entry point of the executable. It sets up the simulation, runs the Monte Carlo loop, computes the approximation, and prints results.

```fortran
program compute_pi_serial
  use, intrinsic :: iso_fortran_env, only : DP => REAL64, I8 => INT64
  use lcgenerator_mod,  only : lcgrandom
  use get_nr_iters_mod, only : get_nr_iters
  
  implicit none

  integer(kind=I8) :: num_trials, i = 0, Ncirc = 0
  real(kind=DP) :: pi_computed = 0.0, pi_actual
  real(kind=DP) :: x = 0.0, y = 0.0, r = 1.0
  real(kind=DP) :: r_squared = 0.0

  pi_actual = 2.0_DP * acos(0.0_DP)
  
  r_squared = r*r

  num_trials = get_nr_iters()

  do i = 1, num_trials
    x = lcgrandom()
    y = lcgrandom()
    if ((x*x + y*y) <= r_squared) then
        Ncirc = Ncirc + 1
    end if
  end do

  pi_computed = 4.0*((1.0*Ncirc)/(1.0*num_trials))

  print '(A, I10, A, F25.15)', "After ", num_trials, " loops, Pi = ", pi_computed
  print '(A, F25.15)', "Actual value of Pi = ", pi_actual
  print '(A, F25.15)', "Absolute difference = ", abs(pi_computed - pi_actual)

end program compute_pi_serial
```

#### Breakdown:

1. **Module Imports and Aliases**:
   - `use, intrinsic :: iso_fortran_env, only : DP => REAL64, I8 => INT64`: This imports the intrinsic module `iso_fortran_env`, which provides portable kind parameters for data types. `DP` is aliased to `REAL64` (64-bit double-precision real), and `I8` to `INT64` (64-bit integer). This ensures cross-platform consistency for high-precision calculations.
   - `use lcgenerator_mod, only : lcgrandom`: Imports the `lcgrandom` function from the `lcgenerator_mod` module for generating pseudo-random numbers.
   - `use get_nr_iters_mod, only : get_nr_iters`: Imports the `get_nr_iters` function from the `get_nr_iters_mod` module to determine the number of trials.
   - `implicit none`: A good practice in Fortran to prevent undeclared variables; all variables must be explicitly declared.

2. **Variable Declarations**:
   - `integer(kind=I8) :: num_trials, i = 0, Ncirc = 0`: Declares 64-bit integers:
     - `num_trials`: Total number of Monte Carlo trials (points to generate).
     - `i`: Loop counter, initialized to 0.
     - `Ncirc`: Counter for points inside the quarter-circle, initialized to 0.
   - `real(kind=DP) :: pi_computed = 0.0, pi_actual`: Double-precision reals:
     - `pi_computed`: The approximated value of π.
     - `pi_actual`: The "true" value of π (computed using `acos`).
   - `real(kind=DP) :: x = 0.0, y = 0.0, r = 1.0`: Coordinates and radius:
     - `x`, `y`: Pseudo-random coordinates in [0,1).
     - `r`: Radius of the circle (fixed at 1.0 for a unit circle).
   - `real(kind=DP) :: r_squared = 0.0`: Precomputed `r*r` (1.0) to avoid repeated multiplication in the loop for minor efficiency.

3. **Computing the Actual π**:
   - `pi_actual = 2.0_DP * acos(0.0_DP)`: Uses the intrinsic `acos` function (arc cosine) to compute π. `acos(0.0)` returns π/2, so multiplying by 2 gives π. The `_DP` suffix ensures double-precision literals.

4. **Precomputing r_squared**:
   - `r_squared = r*r`: Stores 1.0 for use in the distance check inside the loop.

5. **Getting Number of Trials**:
   - `num_trials = get_nr_iters()`: Calls the function from `get_nr_iters_mod` to read the number of iterations. If no command-line argument is provided, it defaults to 1000.

6. **Monte Carlo Loop**:
   - `do i = 1, num_trials`: A simple do-loop that runs `num_trials` times.
     - `x = lcgrandom()`: Generates a pseudo-random number in [0,1) for the x-coordinate.
     - `y = lcgrandom()`: Same for y-coordinate.
     - `if ((x*x + y*y) <= r_squared) then`: Checks if the point (x,y) is inside the quarter-circle by testing if the squared distance from origin (x² + y²) ≤ 1.0.
       - If true, `Ncirc = Ncirc + 1`: Increments the counter.
   - This implements the core Monte Carlo method: the probability a random point in the unit square falls inside the quarter-circle is π/4, so π ≈ 4 * (Ncirc / num_trials).

7. **Computing the Approximation**:
   - `pi_computed = 4.0*((1.0*Ncirc)/(1.0*num_trials))`: Calculates the estimate. The `1.0*` ensures real division (avoiding integer truncation). Note: In modern Fortran, this could be simplified to `4.0 * real(Ncirc, DP) / real(num_trials, DP)`, but the code uses explicit promotion.

8. **Output**:
   - `print '(A, I10, A, F25.15)', "After ", num_trials, " loops, Pi = ", pi_computed`: Prints the result with 10 digits for the integer and 15 decimal places for the float.
   - Similar prints for the actual π and absolute difference (`abs(pi_computed - pi_actual)`).
   - Format specifiers: `A` for strings, `I10` for integers (width 10), `F25.15` for floats (width 25, 15 decimals).

This main program is straightforward and efficient for small `num_trials`, but for large values (e.g., millions), the approximation improves, though runtime increases linearly.

### Module: `get_nr_iters_mod`

This module provides a function to parse the number of iterations from command-line arguments, with error handling and a default value.

```fortran
module get_nr_iters_mod
    implicit none
    private
    public :: get_nr_iters

contains

    function get_nr_iters() result(nr_iters)
        use, intrinsic :: iso_fortran_env, only : error_unit
        use, intrinsic :: iso_fortran_env, only : DP => REAL64, I8 => INT64
        implicit none
        integer(kind=I8) :: nr_iters
        integer(kind=I8), parameter :: default_nr_iters = 1000_I8
        character(len=1024) :: buffer, msg
        integer :: istat

        if (command_argument_count() >= 1) then
            call get_command_argument(1, buffer)
            read (buffer, fmt=*, iostat=istat, iomsg=msg) nr_iters
            if (istat /= 0) then
                write (unit=error_unit, fmt='(2A)') &
                    'error: ', msg
                stop 1
            end if
        else
            nr_iters = default_nr_iters
        end if
    end function get_nr_iters

end module get_nr_iters_mod
```

#### Breakdown:

1. **Module Structure**:
   - `implicit none`: No implicit typing.
   - `private`: All entities are private by default.
   - `public :: get_nr_iters`: Only the function is exposed.

2. **Function `get_nr_iters`**:
   - Returns an `integer(kind=I8)`: The number of iterations.
   - Imports: Re-imports `iso_fortran_env` for `error_unit` (standard error stream) and types.
   - `integer(kind=I8), parameter :: default_nr_iters = 1000_I8`: Constant default value with `_I8` kind suffix.
   - Variables: `buffer` (string to hold argument), `msg` (error message), `istat` (I/O status).

3. **Logic**:
   - `if (command_argument_count() >= 1) then`: Checks if at least one command-line argument exists (using intrinsic `command_argument_count`).
     - `call get_command_argument(1, buffer)`: Retrieves the first argument into `buffer`.
     - `read (buffer, fmt=*, iostat=istat, iomsg=msg) nr_iters`: Parses the string as an integer (free-format `*`).
       - If `istat /= 0` (error), prints to stderr and stops with exit code 1.
   - Else: Uses default 1000.
   
This makes the program flexible: run as `./compute_pi_serial 1000000` for 1 million trials.

### Module: `lcgenerator_mod`

This module implements a basic linear congruential generator (LCG) for pseudo-random numbers in [0,1).

```fortran
module lcgenerator_mod
  use, intrinsic :: iso_fortran_env, only : DP => REAL64, I8 => INT64
  implicit none
  private
  public :: lcgrandom

  contains
    real function lcgrandom()
      integer(kind=I8), parameter :: MULTIPLIER = 1366
      integer(kind=I8), parameter :: ADDEND = 150889
      integer(kind=I8), parameter :: PMOD = 714025
      integer(kind=I8), save :: random_last = 0

      integer(kind=I8) :: random_next = 0
      random_next = mod((MULTIPLIER * random_last + ADDEND), PMOD)
      random_last = random_next
      lcgrandom = (1.0_DP * random_next) / PMOD
      return
    end function lcgrandom
end module lcgenerator_mod
```

#### Breakdown:

1. **Module Structure**:
   - Imports types from `iso_fortran_env`.
   - `implicit none`, `private`, `public :: lcgrandom`: Similar to the previous module.

2. **Function `lcgrandom`**:
   - Returns a `real` (implicitly double-precision via context).
   - Parameters: LCG constants (multiplier a=1366, increment c=150889, modulus m=714025). These are from older literature (e.g., IBM RANDU-like, but custom).
   - `integer(kind=I8), save :: random_last = 0`: Static variable (via `save`) to hold the previous state (seed). Initializes to 0 on first call.

3. **Logic**:
   - `random_next = mod((MULTIPLIER * random_last + ADDEND), PMOD)`: Core LCG formula: X_{n+1} = (a * X_n + c) mod m.
     - Uses `mod` for modulo operation.
   - `random_last = random_next`: Updates the state for the next call.
   - `lcgrandom = (1.0_DP * random_next) / PMOD`: Normalizes to [0,1) by dividing by m (promoting to real).
   - `return`: Explicit return (optional in Fortran).

This LCG is simple and deterministic (same sequence every run due to fixed seed). It's not cryptographically secure or highly random but sufficient for this simulation. For better randomness, modern codes might use Fortran's `random_number` intrinsic, but this demonstrates a custom generator.

### Key Concepts and Notes

- **Monte Carlo Method**: Statistical sampling to approximate π. Accuracy improves with more trials (O(1/sqrt(N)) convergence), but it's probabilistic—results vary per run due to randomness.
- **Fortran-Specifics**: 
  - Modules promote modularity and encapsulation.
  - Kind parameters ensure portability.
  - No explicit main; the `program` block is the executable.
- **Compilation/Running**: Compile with `gfortran compute_pi_serial.f90` (assuming all in one file). Run with optional argument for trials.
- **Limitations**: LCG period is at most m=714025, so for very large `num_trials`, it repeats. Not thread-safe (due to shared `save` variable).
- **Extensions**: Could be parallelized with OpenMP for multi-core speedup.

This code is educational, demonstrating numerical methods, I/O, and modular design in Fortran.

## Code Analysis by ChatGPT 5 Thinking

### What the code does (high-level)

This is a **serial Monte-Carlo estimator for $\pi$**. It repeatedly draws uniform random points $(x,y)\in[0,1)\times[0,1)$ and counts how many fall inside the quarter of a unit circle, i.e., where $x^2+y^2\le 1$. If $N$ points are drawn and $N_{\text{circ}}$ are inside, the estimator is

$$
\widehat{\pi}=4\,\frac{N_{\text{circ}}}{N}.
$$

Two small modules support the program:

* `lcgenerator_mod.f90`: a tiny **linear congruential generator (LCG)** returning a uniform variate in $[0,1)$.
* `get_nr_iters_mod.f90`: parses the **first command-line argument** as the number of trials (with a default if omitted).

Below I walk through each file in detail, explain the Fortran choices (kinds, intrinsics, I/O), and then point out **two important correctness/quality issues** and offer drop-in fixes.

---

### File-by-file explanation

#### 1) `compute_pi_serial.f90` — main program

##### Modules and kinds

```fortran
program compute_pi_serial
  use, intrinsic :: iso_fortran_env, only : DP => REAL64, I8 => INT64
  use lcgenerator_mod,  only : lcgrandom
  use get_nr_iters_mod, only : get_nr_iters
```

* `iso_fortran_env` provides **portably sized kinds**. Here:

  * `DP` aliases `REAL64` (IEEE double precision),
  * `I8` aliases `INT64` (signed 64-bit integer).
* Importing just the names you use (`only：…`) keeps the namespace clean and the intent explicit.

##### Declarations and initialization

```fortran
  integer(kind=I8) :: num_trials, i = 0, Ncirc = 0
  real(kind=DP) :: pi_computed = 0.0, pi_actual
  real(kind=DP) :: x = 0.0, y = 0.0, r = 1.0
  real(kind=DP) :: r_squared = 0.0

  pi_actual = 2.0_DP * acos(0.0_DP)
  r_squared = r*r
```

* `num_trials` is the total number of Monte-Carlo samples (draws).
* `Ncirc` counts hits inside the quarter circle.
* `pi_actual` is computed as $2\arccos(0)=\pi$ in double precision; this avoids hard-coding a constant and is **portable** across compilers.
* `r=1` and `r_squared=1` define the unit circle.

> **Minor omission to fix:** the program should *set* `num_trials` by calling `get_nr_iters()` before the loop (see “Corrections & improvements”).

##### Core Monte-Carlo loop

```fortran
  do i = 1, num_trials
    x = lcgrandom()
    y = lcgrandom()
    if ((x*x + y*y) <= r_squared) then
        Ncirc = Ncirc + 1
    end if
  end do
```

* Each iteration draws two independent uniform(0,1) variates.
* The predicate `x*x + y*y <= 1.0` tests membership in the quarter circle.
* Work and memory are **$O(N)$** and **$O(1)$** respectively.

##### Estimator and output

```fortran
  pi_computed = 4.0*((1.0*Ncirc)/(1.0*num_trials))

  print '(A, I10, A, F25.15)', "After ", num_trials, " loops, Pi = ", pi_computed
  print '(A, F25.15)', "Actual value of Pi = ", pi_actual
  print '(A, F25.15)', "Absolute difference = ", abs(pi_computed - pi_actual)
```

* The estimator is $4\cdot N_{\text{circ}}/N$.
* Formatting:

  * `'I10'` prints a right-justified integer (width 10).
  * `'F25.15'` prints a real number with total width 25 and 15 digits after the decimal point.

> **Type-precision note:** literals `4.0` and `1.0` are **default real** (usually 32-bit). The arithmetic will still be upcast when stored into a `REAL64` variable, but you lose precision in the intermediate steps. Prefer the `_DP` suffix and/or explicit `real(...,kind=DP)` casts (see fixes below).

---

#### 2) `lcgenerator_mod.f90` — Linear congruential generator

```fortran
module lcgenerator_mod
  use, intrinsic :: iso_fortran_env, only : DP => REAL64, I8 => INT64
  implicit none
  private
  public :: lcgrandom
```

* `private` by default shields internal names; only the function `lcgrandom` is exported.

```fortran
contains
  real function lcgrandom()
    integer(kind=I8), parameter :: MULTIPLIER = 1366
    integer(kind=I8), parameter :: ADDEND = 150889
    integer(kind=I8), parameter :: PMOD = 714025
    integer(kind=I8), save :: random_last = 0
```

* This encodes the classic LCG recurrence

  $$
  X_{n+1} = (a X_n + c) \bmod m
  $$

  with $a=1366$, $c=150{,}889$, $m=714{,}025$.
* `save` makes `random_last` **static state** across calls.

```fortran
    integer(kind=I8) :: random_next = 0
    random_next = mod((MULTIPLIER * random_last + ADDEND), PMOD)
    random_last = random_next
    lcgrandom = (1.0_DP * random_next) / PMOD
    return
  end function lcgrandom
end module lcgenerator_mod
```

* The scaled output $\in[0,1)$ is computed as `random_next / PMOD`.
* Using `1.0_DP * random_next` forces the numerator to `REAL64`. However, **the function itself is declared as default `real`**, so the result is immediately **down-cast** to single precision upon return. See the fix below.

**Quality and period.** With $m=714{,}025$ the **maximum period is $m$**; after \~7×10^5 calls the sequence repeats. For large Monte-Carlo runs this is weak. Also, these parameters are not among the better-studied modern LCGs. This is fine pedagogically, but I would not use it for serious statistical work.

**Reproducibility & seeding.** The generator **always starts at 0**, hence it is fully deterministic and reproducible across runs and compilers. You might want to add a seeding interface if you want variability.

**Thread safety.** The `save`d global state means **not thread-safe**. It’s fine here (the program is serial).

---

#### 3) `get_nr_iters_mod.f90` — parse number of trials

```fortran
module get_nr_iters_mod
  implicit none
  private
  public :: get_nr_iters
contains

  function get_nr_iters() result(nr_iters)
    use, intrinsic :: iso_fortran_env, only : error_unit
    use, intrinsic :: iso_fortran_env, only : DP => REAL64, I8 => INT64
    implicit none
    integer(kind=I8) :: nr_iters
    integer(kind=I8), parameter :: default_nr_iters = 1000_I8
    character(len=1024) :: buffer, msg
    integer :: istat
```

* The interface is simple: **no arguments**; returns an `INT64` count.
* A default of **1000** iterations is provided.

```fortran
    if (command_argument_count() >= 1) then
        call get_command_argument(1, buffer)
        read (buffer, fmt=*, iostat=istat, iomsg=msg) nr_iters
        if (istat /= 0) then
            write (unit=error_unit, fmt='(2A)') 'error: ', msg
            stop 1
        end if
    else
        nr_iters = default_nr_iters
    end if
  end function get_nr_iters
end module get_nr_iters_mod
```

* Uses **deferred format read** (`fmt=*`) to parse the first argument into `nr_iters`.
* On parse failure, it writes a helpful message to the **standard error unit** and exits with non-zero status (`stop 1`).

---

### Statistical properties of the estimator (why it works)

Let $Z_i=\mathbf{1}\{X_i^2+Y_i^2\le 1\}$ be i.i.d. Bernoulli with $p=\Pr(Z=1)=\text{Area}( \{x^2+y^2\le 1\}\cap[0,1]^2)=\pi/4$. Then

$$
\widehat{\pi}=4\,\bar Z,\quad
\mathbb{E}[\widehat{\pi}]=\pi,\quad
\operatorname{Var}(\widehat{\pi})=\frac{16}{N}p(1-p)\approx\frac{2.6968}{N}.
$$

So the **standard deviation** is $\approx 1.642/\sqrt{N}$. As rules of thumb:

* $N=10^3$ ⇒ typical absolute error $\sim 5.2\times 10^{-2}$.
* $N=10^6$ ⇒ $\sim 1.6\times 10^{-3}$.

Monte-Carlo converges as $O(N^{-1/2})$; parallelizing increases $N$ faster, but you cannot beat this rate without variance-reduction tricks.

---

### Corrections & precision improvements (recommended)

There are three easy fixes that make the program numerically cleaner and compile-ready.

#### A) Set `num_trials` (and remove the stray `...` line)

Your `compute_pi_serial.f90` contains a literal line with `...` between `r_squared = r*r` and the loop. That will not compile. Replace that region with a call to `get_nr_iters()`:

```fortran
  r_squared = r*r

  ! Parse N from the command line (default 1000 if absent)
  num_trials = get_nr_iters()
```

#### B) Keep all arithmetic in double precision

* Make the RNG return `REAL64`.
* Use double-precision literals and explicit casts in the estimator.

**In `lcgenerator_mod.f90`:**

```fortran
  real(kind=DP) function lcgrandom()
    ...
    lcgrandom = real(random_next, kind=DP) / real(PMOD, kind=DP)
```

**In `compute_pi_serial.f90`:**

```fortran
  pi_computed = 4.0_DP * real(Ncirc, kind=DP) / real(num_trials, kind=DP)
```

This avoids any intermediate rounding to single precision.

#### C) Optional: guard against bad inputs

You can reject non-positive `num_trials`:

```fortran
  if (num_trials <= 0_I8) then
    write(*,*) "num_trials must be positive"
    stop 1
  end if
```

---

### Suggested polished version of `compute_pi_serial.f90`

```fortran
program compute_pi_serial
  use, intrinsic :: iso_fortran_env, only : DP => REAL64, I8 => INT64
  use lcgenerator_mod,  only : lcgrandom
  use get_nr_iters_mod, only : get_nr_iters
  implicit none

  integer(I8) :: num_trials, i, Ncirc
  real(DP)    :: pi_computed, pi_actual
  real(DP)    :: x, y, r, r_squared

  ! Initialize
  Ncirc      = 0_I8
  pi_computed= 0.0_DP
  pi_actual  = 2.0_DP * acos(0.0_DP)     ! = π
  r          = 1.0_DP
  r_squared  = r*r
  num_trials = get_nr_iters()

  if (num_trials <= 0_I8) stop "num_trials must be positive"

  do i = 1_I8, num_trials
    x = lcgrandom()
    y = lcgrandom()
    if (x*x + y*y <= r_squared) Ncirc = Ncirc + 1_I8
  end do

  pi_computed = 4.0_DP * real(Ncirc,kind=DP) / real(num_trials,kind=DP)

  print '(A, I12, A, F25.15)', "After ", num_trials, " loops, Pi = ", pi_computed
  print '(A, F25.15)',         "Actual value of Pi = ", pi_actual
  print '(A, F25.15)',         "Absolute difference = ", abs(pi_computed - pi_actual)
end program compute_pi_serial
```

And in `lcgenerator_mod.f90`, change the function line to:

```fortran
  real(kind=DP) function lcgrandom()
```

---

### How to build and run

```bash
# Build (gfortran example)
gfortran -std=f2008 -O3 lcgenerator_mod.f90 get_nr_iters_mod.f90 compute_pi_serial.f90 -o compute_pi_serial

# Run with 1,000,000 samples
./compute_pi_serial 1000000
```

You should see an estimate within \~$1.6\times 10^{-3}$ of $\pi$ on a typical run (with this LCG it will be deterministic per the current code).

---

### Further remarks & optional enhancements

* **RNG quality/period.** The modulus $m=714{,}025$ caps the period at $m$. If you run more draws than that, the sequence repeats. For higher-quality randomness, either use the Fortran intrinsic `random_number` (simple, portable) or a 48- or 64-bit LCG/PCG with well-vetted parameters.
* **Seeding.** Add a `seed_lcg(seed)` routine that sets `random_last`, and accept a 2nd CLI argument to seed runs.
* **Performance.** In tight loops, the `mod` on 64-bit integers is the main cost. For serious speed you might:

  * keep `random_last` and constants in 64-bit and choose $m=2^{k}$ so modulo is a mask, or
  * batch-generate variates and test multiple points per iteration (SIMD).
* **Parallelization.** The serial algorithm is embarrassingly parallel. If you later add OpenMP/MPI, ensure each thread/process gets an **independent RNG stream** (e.g., leapfrogging or distinct seeds with skip-ahead).

---

If you’d like, I can produce a version that uses the standard `random_number` (for quality) and a command-line seed, plus a tiny unit test to verify the variance scaling.

The above program and modules are compiled and run using Fortran Package Manager (fpm):

## Build the Program using FPM (Fortran Package Manager)

In [2]:
import os
root_dir = ""
root_dir = os.getcwd()

The following configuration file (fpm.toml) was used:

```{literalinclude} code/fpm.toml
---
language: toml
---
```

In [5]:
code_dir = root_dir + "/" + "code"

In [6]:
os.chdir(code_dir)

In [7]:
build_status = os.system("fpm build 2>/dev/null")

## Run the Program using FPM (Fortran Package Manager)

### 10 Iterations

In [8]:
%%timeit -r 10 -n 1
exec_status = os.system("fpm run -- 10 2>/dev/null > output.txt")

80.3 ms ± 26.5 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


In [9]:
%cat output.txt

After         10 loops, Pi =         3.200000047683716
Actual value of Pi =         3.141592653589793
Absolute difference =         0.058407394093923


### 100 Iterations

In [7]:
%%timeit -r 10 -n 1
exec_status = os.system("fpm run -- 100 2>/dev/null > output.txt")

33.5 ms ± 7.69 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


In [8]:
%cat output.txt

After        100 loops, Pi =         3.160417031779042
Actual value of Pi =         3.141592653589793
Absolute difference =         0.018824378189249


### 1,000 Iterations

In [9]:
%%timeit -r 10 -n 1
exec_status = os.system("fpm run -- 1000 2>/dev/null > output.txt")

32.9 ms ± 4.03 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


In [10]:
%cat output.txt

After       1000 loops, Pi =         3.143555466911022
Actual value of Pi =         3.141592653589793
Absolute difference =         0.001962813321229


### 10,000 Iterations

In [11]:
%%timeit -r 10 -n 1
exec_status = os.system("fpm run -- 10000 2>/dev/null > output.txt")

31.8 ms ± 3 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


In [12]:
%cat output.txt

After      10000 loops, Pi =         3.141791477611602
Actual value of Pi =         3.141592653589793
Absolute difference =         0.000198824021809


### 100,000 Iterations

In [13]:
%%timeit -r 10 -n 1
exec_status = os.system("fpm run -- 100000 2>/dev/null > output.txt")

34.4 ms ± 4.95 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


In [14]:
%cat output.txt

After     100000 loops, Pi =         3.141612616406209
Actual value of Pi =         3.141592653589793
Absolute difference =         0.000019962816416


### 1,000,000 Iterations

In [15]:
%%timeit -r 10 -n 1
exec_status = os.system("fpm run -- 1000000 2>/dev/null > output.txt")

35.9 ms ± 5.26 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


In [16]:
%cat output.txt

After    1000000 loops, Pi =         3.141594652402481
Actual value of Pi =         3.141592653589793
Absolute difference =         0.000001998812688


### 10,000,000 Iterations

In [17]:
%%timeit -r 10 -n 1
exec_status = os.system("fpm run -- 10000000 2>/dev/null > output.txt")

64.8 ms ± 7.7 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


In [18]:
%cat output.txt

After   10000000 loops, Pi =         3.141592854147901
Actual value of Pi =         3.141592653589793
Absolute difference =         0.000000200558108


### 100,000,000 Iterations

In [19]:
%%timeit -r 10 -n 1
exec_status = os.system("fpm run -- 100000000 2>/dev/null > output.txt")

274 ms ± 9.74 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


In [20]:
%cat output.txt

After  100000000 loops, Pi =         3.141592668260093
Actual value of Pi =         3.141592653589793
Absolute difference =         0.000000014670300


### 1,000,000,000 Iterations

In [21]:
%%timeit -r 10 -n 1
exec_status = os.system("fpm run -- 1000000000 2>/dev/null > output.txt")

2.34 s ± 46.8 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


In [22]:
%cat output.txt

After 1000000000 loops, Pi =         3.141592665675041
Actual value of Pi =         3.141592653589793
Absolute difference =         0.000000012085248
